<a href="https://colab.research.google.com/github/zyberg2091/e-safety-for-youths/blob/master/Toxic%20Page%20detection/Toxicity_check_page.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd


from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model

In [ ]:

def custom_model(bert_base_model):
    input_word_ids = Input(shape=(200,), dtype=tf.int32)
    sequence_output = bert_base_model(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    X = tf.keras.layers.BatchNormalization()(cls_token)
    X = tf.keras.layers.Dense(200, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)
    out = Dense(2, activation='softmax')(X)

    return tf.keras.Model(inputs=input_word_ids,outputs=out)

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.8MB 5.4MB/s 
     |████████████████████████████████| 2.9MB 16.8MB/s 
     |████████████████████████████████| 890kB 37.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=7e34061d7cf8638231df028830e328421324ef225f90f93d102a552d5b62c0e3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import TFDistilBertModel, DistilBertConfig
from transformers import BertTokenizer
from tokenizers import BertWordPieceTokenizer
import transformers

In [ ]:
bert_layer= (transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
model=custom_model(bert_layer)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f35e25d0d90> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f35e25d0d90> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [ ]:
for layer in model.layers[:3]:
  layer.trainable=False

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
tf_distil_bert_model (TFDist TFBaseModelOutput(last_hi 134734080 
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 768)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 768)               3072      
_________________________________________________________________
dense (Dense)                (None, 200)               153800    
_________________________________________________________________
dropout_19 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402   

In [ ]:
bert_tokenizer = BertTokenizer('vocab.txt', lowercase=False)
bert_tokenizer

PreTrainedTokenizer(name_or_path='', vocab_size=119547, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
model.load_weights('/content/drive/MyDrive/ML Model weights/Toxicity page detection/Toxicity page weights/multilingual weights/tf_model.h5')

In [ ]:
# import pickle

In [ ]:
# with open('/content/drive/MyDrive/ML Model weights/Toxicity page detection/valid_input_ids.pkl','rb') as f:
#   valid_input_ids=pickle.load(f)


In [ ]:
# y_valid_ids=valid_dataset.iloc[:,3].values

In [ ]:
# valid_input_ids[1]

In [ ]:
# type(valid_input_ids),type(y_valid_ids)

In [ ]:
# x_valid_ids=tf.convert_to_tensor(valid_input_ids)
# y_valid_ids=tf.convert_to_tensor(y_valid_ids)

In [ ]:
# y_valid_ids=tf.one_hot(y_valid_ids,2)

In [ ]:
# AUTO=tf.data.experimental.AUTOTUNE

In [ ]:
# model.fit(valid_dataset_bert.repeat(),steps_per_epoch=int(len(x_valid_ids)/32),epochs=1)

In [ ]:
def tokenize_sentences(sentences, tokenizer, max_seq_len = 200):
    tokenized_sentences = []
    for sentence in sentences:
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len, 
                            truncation= True           # Truncate all sentences.
                    )
        
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

In [ ]:
test_sentences=["you really wanna fuck it...i am going to hell.This bitch is nuts. Who would read a book by a woman","what the fuck man lol......you're are such a hypocrite!","i am a good boy.i really love to sing song watch movies","my hobby is playing and i hate you and share you always","我操你","i fuck you"]
test_sentences_2=["Vasya has recently learned to type and log on to the Internet. He immediately entered a chat room and decided to say hello to everybody. Vasya typed the word s. It is considered that Vasya managed to say hello if several letters can be deleted from the typed word so that it resulted in the word 'hello'. For example, if Vasya types the word 'ahhellllloou', it will be considered that he said hello, and if he types 'hlelo', it will be considered that Vasya got misunderstood and he didn't manage to say hello. Determine whether Vasya managed to say hello by the given word s."]
test_ids=tokenize_sentences(test_sentences,bert_tokenizer)
test_ids_2=tokenize_sentences(test_sentences_2,bert_tokenizer)


In [ ]:
test_ids=pad_sequences(test_ids,maxlen=200,padding='post')
test_ids_2=pad_sequences(test_ids_2,maxlen=200,padding='post')

In [ ]:
bert_tokenizer.tokenize("我操你")

['我', '操', '你']

In [ ]:
test_ids=tf.convert_to_tensor(test_ids)
test_ids_2=tf.convert_to_tensor(test_ids_2)

In [ ]:
y_pred,y_pred_2=model.predict(test_ids[:]),model.predict(test_ids_2[:])

In [ ]:
bert_tokenizer("you really wanna fuck it") # this kinda it also works but i created a function tokenize_sentences

{'input_ids': [101, 13028, 30181, 11471, 16441, 11005, 11263, 10271, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
print(y_pred,y_pred_2)

[[4.3731585e-10 1.0000000e+00]
 [7.6992762e-15 1.0000000e+00]
 [1.0000000e+00 2.6965861e-09]
 [9.9999976e-01 2.1803382e-07]
 [9.9999988e-01 8.2955438e-08]
 [5.3848970e-07 9.9999940e-01]] [[1.000000e+00 5.130522e-09]]


In [ ]:
print(y_pred.argmax(-1),y_pred_2.argmax(-1))

[1 1 0 0 0 1] [0]


Now the model can be loaded using the below methods easily

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [ ]:
model_name='zyberg2091/distilbert-base-multilingual-toxicity-classifier'

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

Some layers from the model checkpoint at zyberg2091/distilbert-base-multilingual-toxicity-classifier were not used when initializing TFDistilBertForSequenceClassification: ['input_1', 'dense', 'dropout_38', 'dense_1', 'tf_distil_bert_model_1', 'batch_normalization', 'tf_op_layer_strided_slice']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at zyberg2091/distilbert-base-multilingual-toxicity-classifier and are newly initialized: ['dis

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained(model_name)